<a href="https://colab.research.google.com/github/elias-mena/Uber-Data/blob/main/trips.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd '/content/drive/My Drive/Colab Notebooks/Uber'
!ls

/content/drive/My Drive/Colab Notebooks/Uber
eats.ipynb		eats_restaurant_names.csv  trips.ipynb
eats_order_details.csv	trips_data.csv


In [ ]:
import pandas as pd 
import numpy as np
import plotly.express as px # Interactive plots
from datetime import datetime

## Exploring the data set

Original data frame from Uber

In [ ]:
df = pd.read_csv('trips_data.csv')
df.shape

(176, 15)

In [ ]:
df.sample(3)

,City,Product Type,Trip or Order Status,Request Time,Begin Trip Time,Begin Trip Lat,Begin Trip Lng,Begin Trip Address,Dropoff Time,Dropoff Lat,Dropoff Lng,Dropoff Address,Distance (miles),Fare Amount,Fare Currency
26,781,UberX,COMPLETED,2019-12-26 01:33:29 +0000 UTC,2019-12-26 01:37:15 +0000 UTC,9.965916,-84.026655,"XX8F+99 San José Province, Ipís de Goicoechea,...",2019-12-26 01:51:04 +0000 UTC,9.945303,-84.070549,"100, Colonia Florida, San José Province, San J...",3.72,4500.0,CRC
46,781,UberX,COMPLETED,2019-10-30 12:22:39 +0000 UTC,2019-10-30 12:26:29 +0000 UTC,9.951437,-84.048503,"Novacentro, Santa Cecilia, San José, Guadalupe...",2019-10-30 12:37:51 +0000 UTC,9.946175,-84.071247,"C. 17, Pinos, San José, Costa Rica",2.42,1750.0,CRC
5,781,UberX,COMPLETED,2021-06-12 00:14:26 +0000 UTC,2021-06-12 00:18:42 +0000 UTC,9.934700,-84.055950,"Frente a Prival Bank, Piso 8, Alameda, San Jos...",2021-06-12 00:28:00 +0000 UTC,9.939383,-84.046920,"C. la Cruz 33, San José, Mercedes, Costa Rica",1.32,1650.0,CRC


Variables info

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   City                  176 non-null    int64  
 1   Product Type          174 non-null    object 
 2   Trip or Order Status  176 non-null    object 
 3   Request Time          176 non-null    object 
 4   Begin Trip Time       176 non-null    object 
 5   Begin Trip Lat        165 non-null    float64
 6   Begin Trip Lng        165 non-null    float64
 7   Begin Trip Address    164 non-null    object 
 8   Dropoff Time          176 non-null    object 
 9   Dropoff Lat           165 non-null    float64
 10  Dropoff Lng           165 non-null    float64
 11  Dropoff Address       161 non-null    object 
 12  Distance (miles)      176 non-null    float64
 13  Fare Amount           176 non-null    float64
 14  Fare Currency         174 non-null    object 
dtypes: float64(6), int64(1)

## Cleaning the data

We will choose only the completed trips

In [ ]:
df = df[df['Trip or Order Status'] == "COMPLETED"]
df.sample(3)

,City,Product Type,Trip or Order Status,Request Time,Begin Trip Time,Begin Trip Lat,Begin Trip Lng,Begin Trip Address,Dropoff Time,Dropoff Lat,Dropoff Lng,Dropoff Address,Distance (miles),Fare Amount,Fare Currency
29,781,UberX,COMPLETED,2019-12-11 14:10:01 +0000 UTC,2019-12-11 14:10:37 +0000 UTC,9.948053,-84.051358,"Av Central, Santa Cecilia, San José, Guadalupe...",2019-12-11 14:16:15 +0000 UTC,9.945324,-84.070403,"100, Colonia Florida, San José Province, San J...",1.67,1400.0,CRC
55,781,UberX,COMPLETED,2019-06-26 04:34:20 +0000 UTC,2019-06-26 04:38:06 +0000 UTC,9.971023,-84.008775,"C. 153, San José, Guadalupe, Costa Rica",2019-06-26 04:45:00 +0000 UTC,9.966275,-84.026757,"C. 113, San José, Guadalupe, Costa Rica",1.50,1250.0,CRC
62,781,UberX,COMPLETED,2019-04-07 04:59:56 +0000 UTC,2019-04-07 05:01:33 +0000 UTC,9.935636,-84.093728,"De la entrada a Paseo Colón 500 metros oeste, ...",2019-04-07 05:24:14 +0000 UTC,9.965892,-84.026694,1.5 Kilometros al norte de la clínica Jerusale...,6.00,3800.0,CRC


We will choose only these variables for the analysis



In [ ]:
df = df[['Product Type','Fare Amount','Fare Currency','Request Time']]
df.head(6)

,Product Type,Fare Amount,Fare Currency,Request Time
0,UberEATS Marketplace,5455.0,CRC,2021-08-05 04:52:43 +0000 UTC
1,UberX,4700.0,CRC,2021-07-27 12:55:20 +0000 UTC
2,UberX,2150.0,CRC,2021-07-22 23:43:51 +0000 UTC
3,UberEATS Marketplace,3235.0,CRC,2021-07-21 01:57:19 +0000 UTC
4,UberX,1950.0,CRC,2021-06-20 12:31:25 +0000 UTC
5,UberX,1650.0,CRC,2021-06-12 00:14:26 +0000 UTC


Change the name of the colums

In [ ]:
df.rename(columns={'Fare Amount':'Amount','Fare Currency':'Currency','Request Time':'Time'}, inplace=True)
df.head(2)

,Product Type,Amount,Currency,Time
0,UberEATS Marketplace,5455.0,CRC,2021-08-05 04:52:43 +0000 UTC
1,UberX,4700.0,CRC,2021-07-27 12:55:20 +0000 UTC


Look at the variable types, we need to change it to can manage the data properly

In [ ]:
df.dtypes

Product Type     object
Amount          float64
Currency         object
Time             object
dtype: object

In [ ]:
# Categorical variables
df[['Product Type','Currency']] = df[['Product Type','Currency']].astype('category')

# Numeric variables
df[['Amount']] = df[['Amount']].astype('int')

# Date variables
df['Time'] = df['Time'].str[:10]
df['Time'] = pd.to_datetime(df['Time'])

# A new data frame with the clean data
trips_data = df

# Now look at the types in the new data frame
trips_data.dtypes

Product Type          category
Amount                   int64
Currency              category
Time            datetime64[ns]
dtype: object

We need some others colums with the day, month and week

In [ ]:
trips_data['Day'] = trips_data['Time'].dt.strftime("%A")
trips_data['Month'] = trips_data['Time'].dt.strftime("%B")
trips_data['Week'] = trips_data['Time'].dt.strftime("%W")

trips_data.head(3)

,Product Type,Amount,Currency,Time,Day,Month,Week
0,UberEATS Marketplace,5455,CRC,2021-08-05,Thursday,August,31
1,UberX,4700,CRC,2021-07-27,Tuesday,July,30
2,UberX,2150,CRC,2021-07-22,Thursday,July,29


## Plotting the data

### Trips by Product

A temporal data frame to plot the data

In [ ]:
df_product = trips_data.groupby("Product Type")["Product Type"].count().to_frame()
df_product.rename(columns={'Product Type':'Trips'}, inplace=True)
df_product = df_product.reset_index()
df_product

,Product Type,Trips
0,Comfort,1
1,UberEATS Marketplace,4
2,UberX,91
3,uberX,48
4,uberXL,1


In [ ]:
px.bar(df_product, x = "Product Type", y = "Trips",  title = "Trips by Product")

### Monthly trips

In [ ]:
df_monthly_trips = trips_data.groupby("Month")["Month"].count().to_frame()
df_monthly_trips.rename(columns={'Month':'Trips'}, inplace=True)
df_monthly_trips = df_monthly_trips.reset_index()
df_monthly_trips.head(4)

,Month,Trips
0,April,13
1,August,8
2,December,13
3,February,14


Statictics about the monthly trips

In [ ]:
df_monthly_trips['Trips'].describe()

count    12.000000
mean     12.083333
std       2.391589
min       8.000000
25%      11.250000
50%      13.000000
75%      13.250000
max      15.000000
Name: Trips, dtype: float64

Trips by month

In [ ]:
px.bar_polar(df_monthly_trips, r="Trips", theta="Month",   title = "Trips by month")

### Daily Trips

In [ ]:
df_daily_trips = trips_data.groupby("Day")["Day"].count().to_frame()
df_daily_trips.rename(columns={'Day':'Trips'}, inplace=True)
df_daily_trips = df_daily_trips.reset_index()
df_daily_trips

,Day,Trips
0,Friday,28
1,Monday,16
2,Saturday,18
3,Sunday,28
4,Thursday,20
5,Tuesday,16
6,Wednesday,19


Statictics about the monthly trips

In [ ]:
df_daily_trips["Trips"].describe()

count     7.000000
mean     20.714286
std       5.186980
min      16.000000
25%      17.000000
50%      19.000000
75%      24.000000
max      28.000000
Name: Trips, dtype: float64

In [ ]:
px.bar_polar(df_daily_trips, r="Trips", theta="Day",   title = "Trips by day")

### Monthly Spences

In [ ]:
df_monthly_spences = trips_data.groupby("Month")["Amount"].sum().to_frame()
df_monthly_spences = df_monthly_spences.reset_index()
df_monthly_spences.head(4)

,Month,Amount
0,April,29291
1,August,19433
2,December,27547
3,February,25849


Statistics about monthly spences

In [ ]:
df_monthly_spences["Amount"].describe()

count       12.000000
mean     26376.583333
std       5984.003228
min      18337.000000
25%      21180.250000
50%      26165.000000
75%      29529.250000
max      36309.000000
Name: Amount, dtype: float64

Trend on monthly spences

In [ ]:
px.line(df_monthly_spences, x = "Month", y = "Amount",  title = "Monthly Spences")

### Weekly Spences

In [ ]:
df_weekly_spences = trips_data.groupby("Week")["Amount"].sum().to_frame()
df_weekly_spences = df_weekly_spences.reset_index()
df_weekly_spences.head(4)

,Week,Amount
0,00,7550
1,01,2405
2,03,3652
3,04,2786


Statistics about weekly spences

In [ ]:
df_weekly_spences['Amount'].describe()

count       49.000000
mean      6459.571429
std       4149.779427
min       1250.000000
25%       3563.000000
50%       5355.000000
75%       8020.000000
max      21922.000000
Name: Amount, dtype: float64

Trend on weekly spences

In [ ]:
px.line(df_weekly_spences, x = "Week", y = "Amount",  title = "Weekly Spences")

### Fare Amount histogram

In [ ]:
px.histogram(trips_data, x = "Amount",  title = "Fare Amount")